# Transformer (Attention Is All You Need) 구현하기 (2/3)
- [code 참고 : transformer 구현하기(3/3)](https://paul-hyun.github.io/transformer-03/)
- [이론참고 : Attention is all you need 뽀개기](https://pozalabs.github.io/transformer/)

# 0. Settings

In [1]:
import sentencepiece as spm
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import json
import torch.nn.functional as F
from tqdm.notebook import tqdm # 상태바 표시
from Transformer import *

# 1. Model
### 앞서 구현한 Transformer 클래스를 이용하여 Naver 영화리뷰 감정분석 분류 모델 클래스를  정의


- line 12: Encoder input, Decoder input을 입력으로 Transformer 모델을 실행
- line 14 : Transformer 출력의 max 값 구하기
- line 16 : Linear를 실행하여 최종 예측 결과를 만들기

In [34]:
""" naver movie classfication """
class MovieClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = Transformer(self.config)
        self.projection = nn.Linear(self.config.d_hidn, self.config.n_output, bias=False)
    
    def forward(self, enc_inputs, dec_inputs):
        # (bs, n_dec_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs = self.transformer(enc_inputs, dec_inputs)
        # (bs, d_hidn)
        dec_outputs, _ = torch.max(dec_outputs, dim=1)
        # (bs, n_output)
        logits = self.projection(dec_outputs)
        # (bs, n_output), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        return logits, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

# 2. DataSet
### Naver 영화리뷰 감정 분석 데이터 셋
- line 16 : 입력 파일로부터 'label'을 읽어들임
- line 17 : 입력 파일로부터 'doc' token을 읽어 숫자(token id)로 변경
- line 26 : Decoder 입력은 '[BOS]'로 고정

In [35]:
""" 영화 분류 데이터셋 """
class MovieDataSet(torch.utils.data.Dataset):
    def __init__(self, vocab, infile):
        self.vocab = vocab
        self.labels = []
        self.sentences = []

        line_cnt = 0
        with open(infile, "r") as f:
            for line in f:
                line_cnt += 1

        with open(infile, "r") as f:
            for i, line in enumerate(tqdm(f, total=line_cnt, desc=f"Loading {infile}", unit=" lines")):
                data = json.loads(line)
                self.labels.append(data["label"])
                self.sentences.append([vocab.piece_to_id(p) for p in data["doc"]])
    
    def __len__(self):
        assert len(self.labels) == len(self.sentences)
        return len(self.labels)
    
    def __getitem__(self, item):
        return (torch.tensor(self.labels[item]),
                torch.tensor(self.sentences[item]),
                torch.tensor([self.vocab.piece_to_id("[BOS]")]))

### collate_fn
: 배치 단위로 데이터 처리를 위한 collate_fn 함수
- line 5: Encoder inputs의 길이가 같아지도록 짧은 문장에 padding(0)을 추가
    - padding은 이전에 '-pad_id=0' 옵션으로 지정한 값
- line 6: Decoder inputs의 길이가 같아지도록 짧은 문장에 padding(0)을 추가
- line 9: Label은 길이가 1 고정이므로 Stack함수를 이용해 tensor를 만든다

In [36]:
""" movie data collate_fn """
def movie_collate_fn(inputs):
    labels, enc_inputs, dec_inputs = list(zip(*inputs))

    enc_inputs = torch.nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True, padding_value=0)
    dec_inputs = torch.nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True, padding_value=0)

    batch = [
        torch.stack(labels, dim=0),
        enc_inputs,
        dec_inputs,
    ]
    return batch

### DataLoader
- 위에서 정의한 DataSet과 collate_fn을 이용해 학습용(train_loader), 평가용(test_loader) DataLoader를 만듭니다

In [37]:
# vocab loading
vocab_file = "./data/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [39]:
batch_size = 128
train_dataset = MovieDataSet(vocab, "data/ratings_train.json")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=movie_collate_fn)
test_dataset = MovieDataSet(vocab, "data/ratings_test.json")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=movie_collate_fn)

# 3. Evaluate
: 학습된 MovieClassification 모델의 성능을 평가하기 위한 함수로 평가 지표는 Accuracy 사용
- line 12: Encoder input, Decoder input을 입력으로 MovieClassification을 실행
- line 13 : 1번의 결과 중 첫 번째 값이 예측 logits임
- line 14: logits의 최대값의 인덱스 구하기
- line 16 : 위에서 구한 값과 labels의 값이 같은지 비교

In [40]:
""" 모델 epoch 평가 """
def eval_epoch(config, model, data_loader):
    matchs = []
    model.eval()

    n_word_total = 0
    n_correct_total = 0
    with tqdm(total=len(data_loader), desc=f"Valid") as pbar:
        for i, value in enumerate(data_loader):
            labels, enc_inputs, dec_inputs = map(lambda v: v.to(config.device), value)

            outputs = model(enc_inputs, dec_inputs)
            logits = outputs[0]
            _, indices = logits.max(1)

            match = torch.eq(indices, labels).detach()
            matchs.extend(match.cpu())
            accuracy = np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0

            pbar.update(1)
            pbar.set_postfix_str(f"Acc: {accuracy:.3f}")
    return np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0

# 4. Train
: MovieClassification 모델을 학습하기 위한 함수
- line 11: Encoder input, Decoder input을 입력으로 MovieClassification을 실행
- line 12 : 1번의 결과 중 첫 번째 값이 예측 logits임
- line 14: logits값과 labels의 값을 이용해 Loss 계산 
- line 18,19 : loss, optimizer을 이용해 학습

In [41]:
""" 모델 epoch 학습 """
def train_epoch(config, epoch, model, criterion, optimizer, train_loader):
    losses = []
    model.train()

    with tqdm(total=len(train_loader), desc=f"Train {epoch}") as pbar:
        for i, value in enumerate(train_loader):
            labels, enc_inputs, dec_inputs = map(lambda v: v.to(config.device), value)

            optimizer.zero_grad()
            outputs = model(enc_inputs, dec_inputs)
            logits = outputs[0]

            loss = criterion(logits, labels)
            loss_val = loss.item()
            losses.append(loss_val)

            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss_val:.3f} ({np.mean(losses):.3f})")
    return np.mean(losses)


### 학습을 위한 추가적인 내용 선언
1. GPU 사용 여부를 확인
2. 출력 값 개수를 정의 : 부정(0), 긍정(1) -> 2개 출력 값
3. learning_rate 및 학습 epoch 선언

In [42]:
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.n_output = 2
print(config)

learning_rate = 5e-5
n_epoch = 10

{'n_enc_vocab': 8007, 'n_dec_vocab': 8007, 'n_enc_seq': 256, 'n_dec_seq': 256, 'n_layer': 6, 'd_hidn': 256, 'i_pad': 0, 'd_ff': 1024, 'n_head': 4, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12, 'device': device(type='cuda'), 'n_output': 2}


/home/ubuntu/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
learning_rate = 5e-5
n_epoch = 10

### 위에서 선언된 내용을 이용해 학습 실행
1. MovieClassification 생성
2. MovieClassification이 GPU/CPU를 지원하도록 함
3. loss function선언
4. optimizer를 선언
5. 각 epoch 마다 학습
6. 각 epoch 마다 평가

In [53]:
from Transformer import *

In [45]:
config

{'n_enc_vocab': 8007,
 'n_dec_vocab': 8007,
 'n_enc_seq': 256,
 'n_dec_seq': 256,
 'n_layer': 6,
 'd_hidn': 256,
 'i_pad': 0,
 'd_ff': 1024,
 'n_head': 4,
 'd_head': 64,
 'dropout': 0.1,
 'layer_norm_epsilon': 1e-12,
 'device': device(type='cuda'),
 'n_output': 2}

In [54]:
model = MovieClassification(config)
model.to(config.device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses, scores = [], []
for epoch in range(n_epoch):
    loss = train_epoch(config, epoch, model, criterion, optimizer, train_loader)
    score = eval_epoch(config, model, test_loader)

    losses.append(loss)
    scores.append(score)

ValueError: could not broadcast input array from shape (257,85) into shape (257,128)